# 01. Foundations of Agentic Planning

Audience: DS / AI Engineer researching planning solutions for agentic systems.

Goals:
- Understand why planning is used in agent workflows.
- Identify when to use ReAct vs plan-and-execute style orchestration.
- Build a practical decision rule for pattern selection.


In [ ]:
from __future__ import annotations

import json
import os
import math
import random
import statistics
from pathlib import Path


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'README.md').exists() and (candidate / 'main_langgraph.py').exists():
            return candidate
    return start


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
os.chdir(PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
pattern_taxonomy = [
    {
        'pattern': 'ReAct',
        'strength': 'Adaptive under uncertainty',
        'tradeoff': 'High token and loop overhead',
        'best_for': 'Unknown environments and frequent replanning',
    },
    {
        'pattern': 'Plan-and-Execute',
        'strength': 'Clear structure and lower orchestration variance',
        'tradeoff': 'Plan can be stale if environment changes quickly',
        'best_for': 'Research, analysis, and multi-step deterministic tasks',
    },
    {
        'pattern': 'Plan-and-Act with revisions',
        'strength': 'Balanced control with periodic corrections',
        'tradeoff': 'More orchestration complexity than pure planning',
        'best_for': 'Long tasks with moderate uncertainty',
    },
]

for item in pattern_taxonomy:
    print(f"- {item['pattern']}: {item['best_for']}")


In [ ]:
def choose_pattern(task_complexity: int, uncertainty: int, tool_risk: int) -> str:
    """
    Inputs are 1..10 scales.
    - Higher uncertainty pushes toward ReAct.
    - Higher complexity and tool risk push toward planning.
    """
    planning_score = 0.55 * task_complexity + 0.35 * tool_risk - 0.25 * uncertainty
    react_score = 0.60 * uncertainty + 0.20 * task_complexity

    if planning_score >= react_score + 1.0:
        return 'Plan-and-Execute'
    if react_score >= planning_score + 1.0:
        return 'ReAct'
    return 'Plan-and-Act with revisions'


scenarios = [
    ('Market research report', 8, 4, 8),
    ('Open-world browser troubleshooting', 6, 9, 4),
    ('Moderately dynamic workflow', 7, 6, 6),
]

for name, c, u, r in scenarios:
    print(name, '=>', choose_pattern(c, u, r))


In [ ]:
# Sanity checks
assert choose_pattern(9, 2, 9) == 'Plan-and-Execute'
assert choose_pattern(4, 10, 3) == 'ReAct'
print('Sanity checks passed.')


## Notes

Use planning when you need:
- traceability,
- reproducibility,
- and better control over tool usage.
